In [1]:
from cassandra.cluster import Cluster
cluster = Cluster(["demo-db-1", "demo-db-2", "demo-db-3"])
cass = cluster.connect()

In [2]:
# cass.execute("create keyspace banking with replication={'class': 'SimpleStrategy', 'replication_factor': 2};")

In [3]:
cass.execute("use banking")

In [4]:
cass.execute("drop table if exists loans")

In [5]:
cass.execute("""
create table loans(
    bank_id INT,
    bank_name TEXT static,
    loan_id UUID,
    amount INT,
    state TEXT,
    PRIMARY KEY ((bank_id), amount, loan_id)
) WITH CLUSTERING ORDER BY (amount DESC)
""")

In [6]:
print(cass.execute("describe table loans").one().create_statement)

CREATE TABLE banking.loans (
    bank_id int,
    amount int,
    loan_id uuid,
    bank_name text static,
    state text,
    PRIMARY KEY (bank_id, amount, loan_id)
) WITH CLUSTERING ORDER BY (amount DESC, loan_id ASC)
    AND additional_write_policy = '99p'
    AND bloom_filter_fp_chance = 0.01
    AND caching = {'keys': 'ALL', 'rows_per_partition': 'NONE'}
    AND cdc = false
    AND comment = ''
    AND compaction = {'class': 'org.apache.cassandra.db.compaction.SizeTieredCompactionStrategy', 'max_threshold': '32', 'min_threshold': '4'}
    AND compression = {'chunk_length_in_kb': '16', 'class': 'org.apache.cassandra.io.compress.LZ4Compressor'}
    AND memtable = 'default'
    AND crc_check_chance = 1.0
    AND default_time_to_live = 0
    AND extensions = {}
    AND gc_grace_seconds = 864000
    AND max_index_interval = 2048
    AND memtable_flush_period_in_ms = 0
    AND min_index_interval = 128
    AND read_repair = 'BLOCKING'
    AND speculative_retry = '99p';


# Insert Data

In [7]:
cass.execute("""
INSERT INTO loans (bank_id, bank_name)
VALUES (544, 'test')
""")

In [8]:
import pandas as pd
pd.DataFrame(cass.execute("select * from loans"))

,bank_id,amount,loan_id,bank_name,state
0,544,None,None,test,None


In [9]:
# INSERT is really an UPSERT (UPDATE or INSERT)
cass.execute("""
INSERT INTO loans (bank_id, bank_name)
VALUES (544, 'test2')
""")

In [10]:
import pandas as pd
pd.DataFrame(cass.execute("select * from loans"))

,bank_id,amount,loan_id,bank_name,state
0,544,None,None,test2,None


In [11]:
# cass.execute("""
# INSERT INTO loans (bank_id, bank_name, loan_id)
# VALUES (544, 'test2', UUID())
# """)

In [12]:
# INSERT is really an UPSERT (UPDATE or INSERT)
cass.execute("""
INSERT INTO loans (bank_id, amount, loan_id)
VALUES (544, 300, UUID())
""")

In [13]:
pd.DataFrame(cass.execute("select * from loans"))

,bank_id,amount,loan_id,bank_name,state
0,544,300,3a31fe94-21b2-4eb7-a33f-0c1041ce781c,test2,None


In [14]:
# NOW() and UUID() both return UUIDs
# NOW() is "more" unique (looks at MAC address, timestamp, sequence number)

In [15]:
cass.execute("""
INSERT INTO loans (bank_id, bank_name, amount, loan_id, state)
VALUES (544, 'mybank', 400, NOW(), 'wi')
""")

In [16]:
pd.DataFrame(cass.execute("select * from loans"))

,bank_id,amount,loan_id,bank_name,state
0,544,400,f289b240-7e5e-11ee-8a52-714f42f16ef6,mybank,wi
1,544,300,3a31fe94-21b2-4eb7-a33f-0c1041ce781c,mybank,None


In [17]:
cass.execute("""
INSERT INTO loans (bank_id, bank_name, amount, loan_id, state)
VALUES (999, 'uwcu', 500, NOW(), 'il')
""")

In [18]:
pd.DataFrame(cass.execute("select * from loans"))

,bank_id,amount,loan_id,bank_name,state
0,544,400,f289b240-7e5e-11ee-8a52-714f42f16ef6,mybank,wi
1,544,300,3a31fe94-21b2-4eb7-a33f-0c1041ce781c,mybank,None
2,999,500,f29d1330-7e5e-11ee-8081-75766f9903bb,uwcu,il


In [19]:
cass.execute("""
CREATE TYPE FullName (first text, last text)
""")

In [20]:
cass.execute("""
alter table loans add (username FullName)
""")

In [21]:
cass.execute("""
INSERT INTO loans (bank_id, bank_name, amount, loan_id, username)
VALUES (999, 'uwcu', 500, NOW(), {first:'Tyler', last:'Caraza-Harter'})
""")

In [25]:
pd.DataFrame(cass.execute("select username, username.first, username.last from loans"))

,username,username_first,username_last
0,None,None,None
1,None,None,None
2,None,None,None
3,"(Tyler, Caraza-Harter)",Tyler,Caraza-Harter


## Prepared Statements

In [26]:
uwcu_insert = cass.prepare("""
INSERT INTO loans (bank_id, bank_name, amount, loan_id, username)
VALUES (999, 'uwcu', ?, NOW(), {first:?, last:?})
""")

In [ ]:
# uwcu_insert.VARIOUS_CONFIG

In [27]:
cass.execute(uwcu_insert, (301, "TestFirst", "TestLast"))

In [28]:
pd.DataFrame(cass.execute("select * from loans"))

,bank_id,amount,loan_id,bank_name,state,username
0,544,400,f289b240-7e5e-11ee-8a52-714f42f16ef6,mybank,wi,None
1,544,300,3a31fe94-21b2-4eb7-a33f-0c1041ce781c,mybank,None,None
2,999,500,f29d1330-7e5e-11ee-8081-75766f9903bb,uwcu,il,None
3,999,500,f419c820-7e5e-11ee-8a52-714f42f16ef6,uwcu,None,"(Tyler, Caraza-Harter)"
4,999,301,a4a72810-7e6c-11ee-8a52-714f42f16ef6,uwcu,None,"(TestFirst, TestLast)"


In [29]:
# GROUP BYs: bank_id, state

In [30]:
pd.DataFrame(cass.execute("""
SELECT bank_id, bank_name, AVG(amount)
FROM loans
GROUP BY bank_id
"""))

,bank_id,bank_name,system_avg_amount
0,544,mybank,350
1,999,uwcu,433


In [37]:
# can only group by partition key (or partition key with some more columns of the primary key)

# pd.DataFrame(cass.execute("""
# SELECT state, AVG(amount)
# FROM loans
# GROUP BY state
# """))

In [38]:
from pyspark.sql import SparkSession
spark = (SparkSession.builder
         .appName("cs544")
         .config('spark.jars.packages', 'com.datastax.spark:spark-cassandra-connector_2.12:3.4.0')
         .config("spark.sql.extensions", "com.datastax.spark.connector.CassandraSparkExtensions")
         .getOrCreate())

:: loading settings :: url = jar:file:/usr/local/lib/python3.10/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.datastax.spark#spark-cassandra-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-d4e7ba9f-6c36-4411-b13a-c9f0ee1cce51;1.0
	confs: [default]
	found com.datastax.spark#spark-cassandra-connector_2.12;3.4.0 in central
	found com.datastax.spark#spark-cassandra-connector-driver_2.12;3.4.0 in central
	found com.datastax.oss#java-driver-core-shaded;4.13.0 in central
	found com.datastax.oss#native-protocol;1.5.0 in central
	found com.datastax.oss#java-driver-shaded-guava;25.1-jre-graal-sub-1 in central
	found com.typesafe#config;1.4.1 in central
	found org.slf4j#slf4j-api;1.7.26 in central
	found io.dropwizard.metrics#metrics-core;4.1.18 in central
	found org.hdrhistogram#HdrHistogram;2.1.12 in central
	found org.reactivestreams#reactive-streams;1.0.3 in central
	found com.github.stephenc.jcip#jcip-annotations;1.0-1 in central
	found com.gith

In [39]:
spark.conf.set("spark.sql.catalog.mycat", "com.datastax.spark.connector.datasource.CassandraCatalog")
spark.conf.set("spark.sql.catalog.mycat.spark.cassandra.connection.host", "demo-db-1,demo-db-2,demo-db-3")

In [40]:
spark.sql("""
SELECT *
FROM mycat.banking.loans
""")

DataFrame[bank_id: int, amount: int, loan_id: string, state: string, username: struct<first:string,last:string>, bank_name: string]

In [41]:
spark.sql("""
SELECT *
FROM mycat.banking.loans
""").toPandas()

,bank_id,amount,loan_id,state,username,bank_name
0,544,400,f289b240-7e5e-11ee-8a52-714f42f16ef6,wi,None,mybank
1,544,300,3a31fe94-21b2-4eb7-a33f-0c1041ce781c,None,None,mybank
2,999,500,f29d1330-7e5e-11ee-8081-75766f9903bb,il,None,uwcu
3,999,500,f419c820-7e5e-11ee-8a52-714f42f16ef6,None,"(Tyler, Caraza-Harter)",uwcu
4,999,301,a4a72810-7e6c-11ee-8a52-714f42f16ef6,None,"(TestFirst, TestLast)",uwcu


In [43]:
spark.sql("""
SELECT state, AVG(amount)
FROM mycat.banking.loans
GROUP BY state
""").toPandas()

,state,avg(amount)
0,None,367.0
1,wi,400.0
2,il,500.0


In [44]:
# TODO: dump to parquet in HDFS, or Hive, or wherever
# spark.sql("""
# SELECT *
# FROM mycat.banking.loans
# """).write.....

DataFrame[bank_id: int, amount: int, loan_id: string, state: string, username: struct<first:string,last:string>, bank_name: string]

In [45]:
# Hash Partitioning Demo: It's Not elastic!

In [46]:
import pandas as pd
import string

In [51]:
df = pd.DataFrame({"letter": list(string.ascii_uppercase)})
df.head()

,letter
0,A
1,B
2,C
3,D
4,E


In [55]:
df["partition-before"] = df["letter"].apply(lambda letter: hash(letter) % 4)
df.head()

,letter,partition-before
0,A,3
1,B,1
2,C,0
3,D,0
4,E,3


In [56]:
df["partition-after"] = df["letter"].apply(lambda letter: hash(letter) % 5)
df.head()

,letter,partition-before,partition-after
0,A,3,2
1,B,1,0
2,C,0,1
3,D,0,4
4,E,3,1


In [60]:
# only 11.5% of the letters stayed with the same partition number
(df["partition-before"] == df["partition-after"]).mean()

0.11538461538461539